In [1]:
import pandas as pd

In [2]:
pohang = pd.read_excel("../data/pohang.xlsx")

In [10]:
pohang = pohang.drop_duplicates(['본문']).reset_index(drop = True).drop("Unnamed: 0", axis = 1)

In [72]:
import re

pattern = re.compile('불법주차&gt;')

print(pattern.search("&lt;장애인 전용구역 불법주차&gt;") == None)
print(pattern.search("&lt;장애인 전용구역 불법주차&gt;랄ㄹ랄라") == None)
print(pattern.search("aa&lt;장애인 전용구역 불법주차&gt;랄ㄹ랄라") == None)

False
False
False


In [73]:
park_idx = pohang['본문'].apply(lambda x : pattern.search(str(x)) != None) # 불법주차 패턴 있는 것
normal_idx = pohang['본문'].apply(lambda x : pattern.search(str(x)) == None) # 불법주차 패턴으로 시작하지 않음

In [77]:
pohang[park_idx].shape

(22515, 59)

In [76]:
pohang[normal_idx].shape

(50296, 59)

In [78]:
normal_pohang = pohang[normal_idx]

In [82]:
from konlpy.tag import Okt

okt = Okt()

In [88]:
from knockknock import discord_sender

url = "https://discord.com/api/webhooks/989774347466047529/c5ZeC8lk3gX8Jq8i77WVuIByZhPyNbTwLPIaY-zQEiH9flKeNcWb6hyUODRUFO0sfnKm"


@discord_sender(webhook_url = url)
def tokenizer(x):
    x = str(x)
    if len(x) < 1000:
        return okt.nouns(x)
    else:
        return ["Too Long"]

In [89]:
tokens = normal_pohang['본문'].apply(lambda x : tokenizer(x))

In [91]:
normal_pohang['tokens'] = tokens

/tmp/ipykernel_13749/4262445335.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  normal_pohang['tokens'] = tokens


In [93]:
normal_pohang.to_csv("../data/pohang_token.csv")